In [ ]:

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt



In [ ]:
submission=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
shops=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
items=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
item_cats=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
train=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test=pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
test.head()

In [ ]:
items.head()

In [ ]:
item_cats.head()

In [ ]:
shops.head()

In [ ]:
submission.head()

In [ ]:
train.head()

In [ ]:
print("shape of train:",train.shape)
print("shape of test:",test.shape)
print("shape of submission:",submission.shape)
print("shape of items:",items.shape)
print("shape of item_cats:",item_cats.shape)
print("shape of shops:",shops.shape)

In [ ]:
subset = ['date','date_block_num','shop_id','item_id','item_cnt_day'] 
print(train.duplicated(subset=subset).value_counts()) 
train.drop_duplicates(subset=subset, inplace=True)
print("shape of train:",train.shape)

In [ ]:
test_shops = test.shop_id.unique() 
test_items = test.item_id.unique() 
train = train[train.shop_id.isin(test_shops)] 
train = train[train.item_id.isin(test_items)] 
print("shape of train:",train.shape)

In [ ]:
fig = plt.figure(figsize=(18,9))
plt.subplots_adjust(hspace=.5)

plt.subplot2grid((3,3), (0,0), colspan = 3)
train['shop_id'].value_counts(normalize=True).plot(kind='bar', alpha=0.7)
plt.title('Shop ID Values in the Training Set (Normalized)')

plt.subplot2grid((3,3), (1,0))
train['item_id'].plot(kind='hist', alpha=0.7)
plt.title('Item ID Histogram')

plt.subplot2grid((3,3), (1,1))
train['item_price'].plot(kind='hist', alpha=0.7, color='red')
plt.title('Item Price Histogram')

plt.subplot2grid((3,3), (1,2))
train['item_cnt_day'].plot(kind='hist', alpha=0.7, color='yellow')
plt.title('Item Count Day Histogram')

plt.subplot2grid((3,3), (2,0), colspan = 3)
train['date_block_num'].value_counts(normalize=True).plot(kind='bar', alpha=0.7)
plt.title('Month (date_block_num) Values in the Training Set (Normalized)')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))

plt.subplot2grid((1,2), (0,0))
train['item_price'].plot(kind='box')
plt.title('Item price')

plt.subplot2grid((1,2), (0,1))
train['item_cnt_day'].plot(kind='box')
plt.title('item_cnt_day')

plt.show()

In [ ]:
train['month']=train['date_block_num']

In [ ]:
train['month']+=1
train['month']%=12
print('shape of train',train.shape)

In [ ]:
train['month']

In [ ]:

train['season']=train['month']


In [ ]:
train['season']//=3

In [ ]:
train['season']=train['season'].replace(0,4)

In [ ]:

train=pd.merge(train,shops,how='left',on='shop_id')

In [ ]:
area = train['shop_name'].apply(lambda x: str.replace(x, '!', '')).apply(lambda x: x.split(' ')[0]) 
train['area'] = pd.Categorical(area).codes 
train.head()

In [ ]:
train.shape

In [ ]:
from itertools import product

block_shop_combi = pd.DataFrame(list(product(np.arange(34), test_shops)), columns=['date_block_num','shop_id']) 
shop_item_combi = pd.DataFrame(list(product(test_shops, test_items)), columns=['shop_id','item_id']) 
all_combi = pd.merge(block_shop_combi, shop_item_combi, on=['shop_id'], how='inner')


In [ ]:
train_base = pd.merge(all_combi, train, on=['date_block_num','shop_id','item_id'], how='left') 
train_base['item_cnt_day'].fillna(0, inplace=True) 
train_grp = train_base.groupby(['date_block_num','shop_id','item_id'])

In [ ]:
train_month = pd.DataFrame(train_grp.agg({'item_cnt_day':['sum','count']})).reset_index() 
train_month.columns = ['date_block_num','shop_id','item_id','item_cnt','item_order']
train_month.head()

In [ ]:
grp = train_month.groupby(['shop_id', 'item_id']) 
train_shop = grp.agg({'item_cnt':['mean','median','std'],'item_order':'mean'}).reset_index() 
train_shop.columns = ['shop_id','item_id','cnt_mean_shop','cnt_med_shop','cnt_std_shop','order_mean_shop'] 
print(train_shop[['cnt_mean_shop','cnt_med_shop','cnt_std_shop']].describe())
train_shop.head()

In [ ]:
price_max = train.groupby(['item_id']).max()['item_price'].reset_index()
price_max.rename(columns={'item_price':'item_max_price'}, inplace=True)
price_max.head()

In [ ]:
price = train.groupby(['item_id']).mean()['item_price'].reset_index()
price.rename(columns={'item_price':'item_mean_price'}, inplace=True)
price.head()

In [ ]:
train_price_dc = pd.merge(price, price_max, on=['item_id'], how='left') 
train_price_dc['discount'] = 1 - (train_price_dc['item_mean_price'] / train_price_dc['item_max_price']) 
train_price_dc.drop('item_max_price', axis=1, inplace=True) 
train_price_dc.head()

In [ ]:
train=pd.merge(train,train_price_dc,how='left',on='item_id')

In [ ]:
train